In [3]:
from clean_tabular import CleanData, CleanImages, MergedData
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, SubsetRandomSampler
import torchvision.transforms as transforms
import re
import multiprocessing
import torchvision
from torchbearer import Trial
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler
from torchbearer.callbacks import TensorBoard
from torch.nn import Module
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
import torch.optim as optim
from pytorch_scratch_classification import Dataset
from torchvision import models, datasets
import copy
import time
import pandas as pd
import numpy as np

from collections import defaultdict, Counter

[nltk_data] Downloading package omw-1.4 to /Users/aj8/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/aj8/opt/anaconda3/envs/scaper_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [108]:

merged = MergedData()

{0: 'Home & Garden ', 1: 'Baby & Kids Stuff ', 2: 'DIY Tools & Materials ', 3: 'Music, Films, Books & Games ', 4: 'Phones, Mobile Phones & Telecoms ', 5: 'Clothes, Footwear & Accessories ', 6: 'Other Goods ', 7: 'Health & Beauty ', 8: 'Sports, Leisure & Travel ', 9: 'Appliances ', 10: 'Computers & Software ', 11: 'Office Furniture & Equipment ', 12: 'Video Games & Consoles '}
{'Home & Garden ': 0, 'Baby & Kids Stuff ': 1, 'DIY Tools & Materials ': 2, 'Music, Films, Books & Games ': 3, 'Phones, Mobile Phones & Telecoms ': 4, 'Clothes, Footwear & Accessories ': 5, 'Other Goods ': 6, 'Health & Beauty ': 7, 'Sports, Leisure & Travel ': 8, 'Appliances ': 9, 'Computers & Software ': 10, 'Office Furniture & Equipment ': 11, 'Video Games & Consoles ': 12}
{0: 'Home & Garden ', 1: 'Baby & Kids Stuff ', 2: 'DIY Tools & Materials ', 3: 'Music, Films, Books & Games ', 4: 'Phones, Mobile Phones & Telecoms ', 5: 'Clothes, Footwear & Accessories ', 6: 'Other Goods ', 7: 'Health & Beauty ', 8: 'Sports

In [109]:
merged.merged_frame

,image_id,image,image_array,image_shape,mode,id,bucket_link,image_ref,create_time_x,product_name,category,product_description,price,location,page_id,create_time_y,major_category,minor_category,major_category_encoded,minor_category_encoded
0,e0ffd840-fb95-4284-b422-edbafb6848fd,e0ffd840-fb95-4284-b422-edbafb6848fd.jpg,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","(224, 224, 3)",RGB,394a4a28-1c8e-4fd7-8342-22803a2b8d8c,https://aicore-product-images.s3.amazonaws.com...,ZHMAAOSwnXxiEn6d,2022-02-27,Fit Bit second hand in Immaculate condition | ...,"Sports, Leisure & Travel / Fitness & Gym Equip...",I have to declare second hand as I’ve attempte...,60.0,"Llantwit Fardre, Rhondda Cynon Taf",1426361345,2022-02-27,"Sports, Leisure & Travel",Fitness & Gym Equipment,8,32
1,7ac78bfb-73db-41e8-b6d1-c9f158995c06,7ac78bfb-73db-41e8-b6d1-c9f158995c06.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(224, 224, 3)",RGB,3c852f17-a4d6-4677-a6f4-e3145987ce3f,https://aicore-product-images.s3.amazonaws.com...,Dd0AAOSw~QRiGQAn,2022-02-28,"PS4 controller | in Sheffield, South Yorkshire...",Video Games & Consoles / Consoles / PS4 (Sony ...,Perfect condition no offers £30Collection or d...,30.0,"Sheffield, South Yorkshire",1426703201,2022-02-28,Video Games & Consoles,Consoles,12,19
2,6c4937e3-b1da-4631-b55f-b80cda35756b,6c4937e3-b1da-4631-b55f-b80cda35756b.jpg,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","(224, 224, 3)",RGB,f217055b-7213-4d89-8e9f-f118d0ff7081,https://aicore-product-images.s3.amazonaws.com...,qKUAAOSwwCNfw-sL,2022-02-26,Gazebo / Marquee for Hire (From £75) | in Croy...,Home & Garden / Garden & Patio / Garden & Pati...,Gazebo / Marquee for hire in and around London...,75.0,"Croydon, London",1391540411,2022-02-26,Home & Garden,Garden & Patio,0,39
3,75da6964-0d3c-407f-9fdb-8d6339c75e04,75da6964-0d3c-407f-9fdb-8d6339c75e04.jpg,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","(224, 224, 3)",RGB,f217055b-7213-4d89-8e9f-f118d0ff7081,https://aicore-product-images.s3.amazonaws.com...,DdUAAOSwbYFfw-sG,2022-02-26,Gazebo / Marquee for Hire (From £75) | in Croy...,Home & Garden / Garden & Patio / Garden & Pati...,Gazebo / Marquee for hire in and around London...,75.0,"Croydon, London",1391540411,2022-02-26,Home & Garden,Garden & Patio,0,39
4,b2d1f493-eb35-4a4a-877a-0089818f83a4,b2d1f493-eb35-4a4a-877a-0089818f83a4.jpg,"[[[1, 1, 1], [230, 230, 230], [227, 227, 227],...","(224, 224, 3)",RGB,a6000ba2-701e-4223-bfc3-27293e6a5e08,https://aicore-product-images.s3.amazonaws.com...,z0oAAOSwIRhiGRG1,2022-02-26,Toys - Playmobil 6687 Super 4 Lost Island with...,Baby & Kids Stuff / Toys,The Super 4 Lost Island with Alien and Raptor ...,6.0,"Bromley, London",1426707628,2022-02-26,Baby & Kids Stuff,Toys,1,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12407,db43dc20-d9f9-41cd-8f0a-24a1496aeba2,db43dc20-d9f9-41cd-8f0a-24a1496aeba2.jpg,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","(224, 224, 3)",RGB,8288d653-7e37-40af-a9d5-fbddec3a17a5,https://aicore-product-images.s3.amazonaws.com...,bJcAAOSwVAhiGRGi,2022-02-27,Dolls house items - dining room | in Musselbur...,"Other Goods / Hobbies, Interests & Collectibles",Dolls house dining room bundle.Buyer to collec...,40.0,"Musselburgh, East Lothian",1426708439,2022-02-27,Other Goods,"Hobbies, Interests & Collectibles",6,48
12408,f79e40c4-9c30-4e8c-9661-c60b969addaf,f79e40c4-9c30-4e8c-9661-c60b969addaf.jpg,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","(224, 224, 3)",RGB,c8f3bb82-6369-42fe-bdc7-d45e505af73c,https://aicore-product-images.s3.amazonaws.com...,0RoAAOSwaJBhvzlN,2022-02-27,Murder One Season 1 DVD Boxset | in South Wood...,"Music, Films, Books & Games / Films & TV / DVDs",Murder One Season 1 DVD Box set. Six disk set ...,5.0,"South Woodham Ferrers, Essex",1422011594,2022-02-27,"Music, Films, Books & Games",Films & TV,3,30
12409,bd0cee0e-563b-4395-81fe-c962719350b2,bd0cee0e-563b-4395-81fe-c962719350b2.jpg,"[[[0, 0, 0], [0, 0, 0], [0, 0,

In [115]:
def embedder(col='product_description'):
    print(merged.merged_frame['product_description'].map(merged.vocab_encoder()[0]))
    print(len(merged.merged_frame['product_description'].map(merged.vocab_encoder()[0])))
    merged.merged_frame['product_description'].map(merged.vocab_encoder()[0])

In [5]:
class WordDataset(Dataset):
    def __init__(self, win_size=3):
        word_cls = MergedData()
        cleaned_product = word_cls.clean_prod()
        self.text = cleaned_product[0]
        self.all_unique_words = cleaned_product[1]
        self.window_size = win_size
        self.vocab_size  = word_cls.get_word_set(col='product_description')[1]
        print(self.vocab_size)
        self.int2wrd = word_cls.vocab_encoder()[1]
        print(self.int2wrd)
        self.wrd2int = word_cls.vocab_encoder()[0]
        print(self.wrd2int)

    def __len__(self):
        return len(self.all_wrds) - self.window_size

    def __getitem__(self, idx):
        X = torch.LongTensor([self.wrd2int[ind] for ind in self.text[idx: idx + self.window_size]])        
        y = self.wrd2int[self.text[idx + self.window_size]]
        return X, y

In [ ]:
class Network(nn.Module):
    def __init__(self, embedding_dim, dense_dim, hidden_dim, n_layers):
        pass

In [4]:
test_sentence = 'the quick brown fox jumped over lazy dog'.split()
CONTEXT_SIZE = 2
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

In [5]:
skip_gram = [
    [
        list(np.array([[test_sentence[i - j], test_sentence[i+j]] for j in range(1, CONTEXT_SIZE+1)]).flatten()),
        test_sentence[i]
    ]
    for i in range(CONTEXT_SIZE, len(test_sentence)-CONTEXT_SIZE)
]

skip_gram

[[['quick', 'fox', 'the', 'jumped'], 'brown'],
 [['brown', 'jumped', 'quick', 'over'], 'fox'],
 [['fox', 'over', 'brown', 'lazy'], 'jumped'],
 [['jumped', 'lazy', 'fox', 'dog'], 'over']]

In [101]:
np.array(skip_gram)[:, 1]

/var/folders/sv/k_x6s66n6ls2xvtk5677t8_r0000gn/T/ipykernel_3663/2460276757.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(skip_gram)[:, 1]


array(['brown', 'fox', 'jumped', 'over'], dtype=object)

In [77]:
skip_gram

AttributeError: 'list' object has no attribute 'to_array'

In [75]:
r1

[[['brown', 'jumped'], ['quick', 'over'], 'fox'],
 [['fox', 'over'], ['brown', 'lazy'], 'jumped'],
 [['jumped', 'lazy'], ['fox', 'dog'], 'over']]

In [49]:
df = pd.DataFrame(columns=['word', 'context'])

In [62]:
l1 = [1,2]
l1 = *'ab'

SyntaxError: can't use starred expression here (2375518790.py, line 2)

In [44]:
print(skip_gram)

[([['brown', 'jumped'], ['quick', 'over']], 'fox'), ([['fox', 'over'], ['brown', 'lazy']], 'jumped')]


In [30]:
first

[([['brown', 'jumped'], ['quick', 'over']],),
 ([['fox', 'over'], ['brown', 'lazy']],),
 ([['jumped', 'lazy'], ['fox', 'dog']],)]